In [1]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import numpy as np
import cv2
import pandas as pd
import os
import math

C:\Users\felip\AppData\Roaming\Python\Python310\site-packages\scipy\__init__.py:169: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=True,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)

In [3]:
images_sad = os.listdir('../face_sad')
images_happy = os.listdir('../face_happy')
images_neutral = os.listdir('../face_neutral')
images_angry = os.listdir('../face_angry')
images_surprised = os.listdir('../face_surprised')
images_disgusted = os.listdir('../face_disgusted')

print(len(images_sad), len(images_happy), len(images_neutral), len(images_angry), len(images_surprised), len(images_disgusted))

8391 8996 10004 7979 7988 7999


In [4]:
def normalize_landmarks(mesh):
    nose_tip = mesh[4]
    forehead = mesh[151]
    scale_factor = math.sqrt(((nose_tip[0]-forehead[0])**2) + ((nose_tip[1]-forehead[1])**2) + ((nose_tip[2]-forehead[2])**2))
    mesh_norm = []
    for point in mesh:
        dist = math.sqrt(((point[0]-forehead[0])**2) + ((point[1]-forehead[1])**2) + ((point[2]-forehead[2])**2))
        point = dist/scale_factor
        mesh_norm.append(point)
    return mesh_norm

In [8]:
def inference(image):
    detection_result = detector.detect(image)
    if detection_result is None or not len(detection_result.face_blendshapes):
        return None
    blendshapes = detection_result.face_blendshapes[0]
    landmarks = detection_result.face_landmarks[0]
    blendshapes_scores = [cat.score for cat in blendshapes]
    mesh_points = np.array([[p.x, p.y, p.z] for p in landmarks])
    normalized_mesh_points = normalize_landmarks(mesh_points)
    return [*blendshapes_scores, *normalized_mesh_points]

In [6]:
def resize_if_too_small(photo):
    if photo.shape[0] < 300 and photo.shape[1] < 300:
        photo = cv2.resize(photo, (300, 300), interpolation = cv2.INTER_CUBIC)
    return photo

In [9]:
def mount_data(image_names, folder):
    data = []
    for image in image_names:
        try:
            cv2image = cv2.imread(folder + image)
            cv2image = resize_if_too_small(cv2image)
            inferenceinput = mp.Image(image_format=mp.ImageFormat.SRGB, data=cv2image)
        except (RuntimeError, AttributeError):
            continue
        inference_result = inference(inferenceinput)
        if inference_result:
            data.append(inference_result)
    print(len(data))
    return data

In [10]:
data_surprised = mount_data(images_surprised, '../face_surprised/')
data_disgusted = mount_data(images_disgusted, '../face_disgusted/')
data_angry = mount_data(images_angry, '../face_angry/')
data_happy = mount_data(images_happy, '../face_happy/')
data_sad = mount_data(images_sad, '../face_sad/')
data_neutral = mount_data(images_neutral, '../face_neutral/')

7050
5190
6764
8475
6708
8247


In [11]:
df_sad = pd.DataFrame(data_sad)
df_happy = pd.DataFrame(data_happy)
df_neutral = pd.DataFrame(data_neutral)
df_angry = pd.DataFrame(data_angry)
df_surprise = pd.DataFrame(data_surprised)
df_disgust = pd.DataFrame(data_disgusted)

In [12]:
df_sad['label'] = 'sad'
df_happy['label'] = 'happy'
df_neutral['label'] = 'neutral'
df_angry['label'] = 'angry'
df_surprise['label'] = 'surprise'
df_disgust['label'] = 'disgust'

df = pd.concat([df_sad, df_happy, df_neutral, df_angry, df_surprise, df_disgust])
df.to_csv('data.csv', index=False)